## Sessionisation

In [1]:
import numpy as np
import pandas as pd
from time import mktime, strptime
from tabulate import tabulate

In [2]:

def divideByIP(log):
    sessions = []
    for ip in log.IP.unique():
        dftemp = log.loc[log.IP==ip, :]
        sessions.append(dftemp)
    return sessions

def divideByH1(log, h1):
    h1 = h1*60
    start = [0, log.Epoch[log.index[0]]]
    sessions = []
    for index, row in log.iterrows():
        if (row['Epoch']-start[1])>h1:
            dftemp = log.loc[start[0]:index-1, :]
            start = [index, row['Epoch']]
            sessions.append(dftemp)
    dftemp = log.loc[start[0]:, :]
    sessions.append(dftemp)
    return sessions

def getSessions(log, h1):
    sessions = []
    for x in divideByIP(log):
        sessions.append(divideByH1(x, h1))
    return sessions

def addEpoch(log, timestr):
    log['Epoch'] = None
    log['Epoch'] = list(map(lambda x: mktime(strptime(x, timestr)), log['Time']))
    return log

def shortenURL(log):
    log['URL'] = list(map(lambda x: x[:30], log['URL']))
    return log

def printSessions(sessions):
    nusers = len(sessions)
    print('Sessions generated for {} users.\n'.format(nusers))
    for user in range(nusers):
        print('*'*100)
        print('Sessions of user #{}'.format(user+1))
        print('*'*100)
        print()
        nsess = len(sessions[user])
        for session in range(nsess):
            print("session no: ",session+1)
            print(tabulate(sessions[user][session].loc[:, ['IP', 'Time', 'URL']], headers=['IP', 'Time', 'URL'], tablefmt='orgtbl'))
            print()
        print()

In [3]:

# Loading and preprocessing log file for question 1
log = pd.read_csv('dataset.csv', header=None)
log.columns=['IP', 'Time', 'URL', 'Status', 'Bytes']
log = shortenURL(log)
timestr = '[%d/%b/%Y:%H:%M:%S%z]'
log = addEpoch(log, timestr)
log.head()

,IP,Time,URL,Status,Bytes,Epoch
0,172.20.112.25,[02/Feb/2000:10:22:02-0500],GET /airmedia/images/welcome.g,200,3081.0,949467122.0
1,172.20.112.25,[02/Feb/2000:10:22:02-0500],GET /airmedia/images/edit_off.,200,268.0,949467122.0
2,172.20.112.25,[02/Feb/2000:10:22:02-0500],GET /airmedia/images/pubconten,200,1388.0,949467122.0
3,172.20.112.25,[02/Feb/2000:10:23:02-0500],GET /airmedia/gateway/1.htmlHT,304,252.0,949467182.0
4,172.20.112.25,[02/Feb/2000:10:23:02-0500],GET /airmedia/gateway/images/6,304,428.0,949467182.0


In [4]:

h1 = 30
sessions = getSessions(log, h1)
printSessions(sessions)

Sessions generated for 2 users.

****************************************************************************************************
Sessions of user #1
****************************************************************************************************

session no:  1
|    | IP            | Time                        | URL                            |
|----+---------------+-----------------------------+--------------------------------|
|  0 | 172.20.112.25 | [02/Feb/2000:10:22:02-0500] | GET /airmedia/images/welcome.g |
|  1 | 172.20.112.25 | [02/Feb/2000:10:22:02-0500] | GET /airmedia/images/edit_off. |
|  2 | 172.20.112.25 | [02/Feb/2000:10:22:02-0500] | GET /airmedia/images/pubconten |
|  3 | 172.20.112.25 | [02/Feb/2000:10:23:02-0500] | GET /airmedia/gateway/1.htmlHT |
|  4 | 172.20.112.25 | [02/Feb/2000:10:23:02-0500] | GET /airmedia/gateway/images/6 |
|  5 | 172.20.112.25 | [02/Feb/2000:10:22:01-0500] | GET /airmedia/images/topic1_on |
|  6 | 172.20.112.25 | [02/Feb/2000:10:22

In [5]:
# Loading log file for question 2
log = pd.read_csv('weblog.csv')
log = shortenURL(log)
log.head()

,IP,Time,URL,Staus
0,10.128.2.1,[29/Nov/2017:06:58:55,GET /login.php HTTP/1.1,200
1,10.128.2.1,[29/Nov/2017:06:59:02,POST /process.php HTTP/1.1,302
2,10.128.2.1,[29/Nov/2017:06:59:03,GET /home.php HTTP/1.1,200
3,10.131.2.1,[29/Nov/2017:06:59:04,GET /js/vendor/moment.min.js H,200
4,10.130.2.1,[29/Nov/2017:06:59:06,GET /bootstrap-3.3.7/js/bootst,200


In [6]:
# Checking for null values
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16007 entries, 0 to 16006
Data columns (total 4 columns):
IP       16007 non-null object
Time     16007 non-null object
URL      16007 non-null object
Staus    16007 non-null object
dtypes: object(4)
memory usage: 500.3+ KB


In [7]:
# Keeping only valid IP addresses
log = log.loc[log.IP.isin(log.IP.unique()[:5])]
log.IP.unique()

array(['10.128.2.1', '10.131.2.1', '10.130.2.1', '10.129.2.1',
       '10.131.0.1'], dtype=object)

In [8]:
# Appending the epoch values
fstring = '[%d/%b/%Y:%H:%M:%S'
log['Epoch'] = list(map(lambda x: mktime(strptime(x, fstring)), log['Time']))
log.head()

,IP,Time,URL,Staus,Epoch
0,10.128.2.1,[29/Nov/2017:06:58:55,GET /login.php HTTP/1.1,200,1.511919e+09
1,10.128.2.1,[29/Nov/2017:06:59:02,POST /process.php HTTP/1.1,302,1.511919e+09
2,10.128.2.1,[29/Nov/2017:06:59:03,GET /home.php HTTP/1.1,200,1.511919e+09
3,10.131.2.1,[29/Nov/2017:06:59:04,GET /js/vendor/moment.min.js H,200,1.511919e+09
4,10.130.2.1,[29/Nov/2017:06:59:06,GET /bootstrap-3.3.7/js/bootst,200,1.511919e+09


In [9]:
# Printing sessions for question 2
h1 = 30
sessions = getSessions(log, h1)
printSessions(sessions)

Sessions generated for 5 users.

****************************************************************************************************
Sessions of user #1
****************************************************************************************************

session no:  1
|    | IP         | Time                  | URL                            |
|----+------------+-----------------------+--------------------------------|
|  0 | 10.128.2.1 | [29/Nov/2017:06:58:55 | GET /login.php HTTP/1.1        |
|  1 | 10.128.2.1 | [29/Nov/2017:06:59:02 | POST /process.php HTTP/1.1     |
|  2 | 10.128.2.1 | [29/Nov/2017:06:59:03 | GET /home.php HTTP/1.1         |
|  6 | 10.128.2.1 | [29/Nov/2017:06:59:19 | GET /js/jquery.min.js HTTP/1.1 |

session no:  2
|    | IP         | Time                  | URL                            |
|----+------------+-----------------------+--------------------------------|
| 21 | 10.128.2.1 | [29/Nov/2017:13:38:20 | GET /css/font-awesome.min.css  |
| 23 | 10.128.2.1 | 

| 4148 | 10.128.2.1 | [02/Dec/2017:16:45:22 | GET /login.php HTTP/1.1        |

session no:  53
|      | IP         | Time                  | URL                            |
|------+------------+-----------------------+--------------------------------|
| 4180 | 10.128.2.1 | [02/Dec/2017:19:18:21 | GET /login.php HTTP/1.1        |
| 4182 | 10.128.2.1 | [02/Dec/2017:19:18:22 | GET /css/font-awesome.min.css  |
| 4195 | 10.128.2.1 | [02/Dec/2017:19:29:39 | GET /contestsubmission.php?id= |
| 4196 | 10.128.2.1 | [02/Dec/2017:19:29:45 | GET /profile.php?user=FFIsty_1 |
| 4197 | 10.128.2.1 | [02/Dec/2017:19:29:46 | GET /js/jquery.min.js HTTP/1.1 |

session no:  54
|      | IP         | Time                  | URL            |
|------+------------+-----------------------+----------------|
| 4207 | 10.128.2.1 | [02/Dec/2017:21:19:11 | GET / HTTP/1.1 |

session no:  55
|      | IP         | Time                  | URL                        |
|------+------------+-----------------------+--------

| 8490 | 10.128.2.1 | [15/Dec/2017:01:40:41 | GET / HTTP/1.1 |

session no:  92
|      | IP         | Time                  | URL                            |
|------+------------+-----------------------+--------------------------------|
| 8492 | 10.128.2.1 | [15/Dec/2017:08:47:55 | GET / HTTP/1.1                 |
| 8493 | 10.128.2.1 | [15/Dec/2017:08:47:55 | GET /login.php HTTP/1.1        |
| 8494 | 10.128.2.1 | [15/Dec/2017:08:47:56 | GET /css/bootstrap.min.css HTT |
| 8497 | 10.128.2.1 | [15/Dec/2017:08:47:56 | GET /css/main.css HTTP/1.1     |
| 8499 | 10.128.2.1 | [15/Dec/2017:08:47:56 | GET /js/vendor/modernizr-2.8.3 |
| 8513 | 10.128.2.1 | [15/Dec/2017:08:55:08 | GET /js/vendor/moment.min.js H |
| 8514 | 10.128.2.1 | [15/Dec/2017:08:55:16 | GET /compiler.php HTTP/1.1     |
| 8515 | 10.128.2.1 | [15/Dec/2017:08:55:16 | GET /archive.php HTTP/1.1      |
| 8516 | 10.128.2.1 | [15/Dec/2017:08:55:20 | GET /contest.php HTTP/1.1      |
| 8517 | 10.128.2.1 | [15/Dec/2017:08:55:23 | GET /

|       | IP         | Time                  | URL                            |
|-------+------------+-----------------------+--------------------------------|
|  9503 | 10.128.2.1 | [29/Jan/2018:20:17:50 | GET /home.php HTTP/1.1         |
|  9505 | 10.128.2.1 | [29/Jan/2018:20:17:50 | GET /login.php HTTP/1.1        |
|  9506 | 10.128.2.1 | [29/Jan/2018:20:17:50 | GET /js/vendor/modernizr-2.8.3 |
|  9507 | 10.128.2.1 | [29/Jan/2018:20:17:50 | GET /home.php HTTP/1.1         |
|  9508 | 10.128.2.1 | [29/Jan/2018:20:17:51 | GET /home.php HTTP/1.1         |
|  9509 | 10.128.2.1 | [29/Jan/2018:20:17:51 | GET /login.php HTTP/1.1        |
|  9510 | 10.128.2.1 | [29/Jan/2018:20:17:51 | GET /js/vendor/modernizr-2.8.3 |
|  9519 | 10.128.2.1 | [29/Jan/2018:20:17:53 | GET /home.php HTTP/1.1         |
|  9520 | 10.128.2.1 | [29/Jan/2018:20:17:53 | GET /home.php HTTP/1.1         |
|  9522 | 10.128.2.1 | [29/Jan/2018:20:17:53 | GET /login.php HTTP/1.1        |
|  9523 | 10.128.2.1 | [29/Jan/2018:20:1

| 15443 | 10.128.2.1 | [25/Feb/2018:06:21:06 | POST /allsubmission.php HTTP/1 |

session no:  204
|       | IP         | Time                  | URL                      |
|-------+------------+-----------------------+--------------------------|
| 15444 | 10.128.2.1 | [25/Feb/2018:07:05:52 | GET /robots.txt HTTP/1.1 |

session no:  205
|       | IP         | Time                  | URL                            |
|-------+------------+-----------------------+--------------------------------|
| 15445 | 10.128.2.1 | [25/Feb/2018:09:33:01 | GET /home.php HTTP/1.1         |
| 15448 | 10.128.2.1 | [25/Feb/2018:09:33:01 | GET /css/font-awesome.min.css  |
| 15450 | 10.128.2.1 | [25/Feb/2018:09:33:02 | GET /css/main.css HTTP/1.1     |
| 15453 | 10.128.2.1 | [25/Feb/2018:09:33:02 | GET /js/vendor/jquery-1.12.0.m |

session no:  206
|       | IP         | Time                  | URL                            |
|-------+------------+-----------------------+--------------------------------|
| 15

...

|      | IP         | Time                  | URL                            |
|------+------------+-----------------------+--------------------------------|
| 4319 | 10.129.2.1 | [03/Dec/2017:15:53:48 | GET / HTTP/1.1                 |
| 4324 | 10.129.2.1 | [03/Dec/2017:15:53:50 | GET /css/main.css HTTP/1.1     |
| 4328 | 10.129.2.1 | [03/Dec/2017:15:53:52 | GET /bootstrap-3.3.7/js/bootst |
| 4329 | 10.129.2.1 | [03/Dec/2017:16:03:16 | GET / HTTP/1.1                 |
| 4330 | 10.129.2.1 | [03/Dec/2017:16:03:16 | GET /login.php HTTP/1.1        |
| 4334 | 10.129.2.1 | [08/Nov/2017:00:39:07 | GET / HTTP/1.1                 |
| 4335 | 10.129.2.1 | [08/Nov/2017:00:39:07 | GET /login.php HTTP/1.1        |
| 4337 | 10.129.2.1 | [08/Nov/2017:01:36:22 | GET /login.php HTTP/1.1        |
| 4338 | 10.129.2.1 | [08/Nov/2017:03:12:46 | GET / HTTP/1.1                 |
| 4342 | 10.129.2.1 | [08/Nov/2017:06:05:20 | GET / HTTP/1.1                 |
| 4343 | 10.129.2.1 | [08/Nov/2017:06:05:21 | GET /l

|      | IP         | Time                  | URL                            |
|------+------------+-----------------------+--------------------------------|
| 4331 | 10.131.0.1 | [03/Dec/2017:17:40:39 | GET /details.php?id=43 HTTP/1. |
| 4339 | 10.131.0.1 | [08/Nov/2017:03:12:46 | GET /login.php HTTP/1.1        |
| 4344 | 10.131.0.1 | [08/Nov/2017:06:52:17 | GET /robots.txt HTTP/1.1       |
| 4345 | 10.131.0.1 | [08/Nov/2017:07:32:26 | GET /robots.txt HTTP/1.1       |
| 4346 | 10.131.0.1 | [08/Nov/2017:07:32:26 | GET / HTTP/1.1                 |
| 4347 | 10.131.0.1 | [08/Nov/2017:07:32:26 | GET /login.php HTTP/1.1        |
| 4350 | 10.131.0.1 | [08/Nov/2017:10:03:30 | GET / HTTP/1.1                 |
| 4354 | 10.131.0.1 | [08/Nov/2017:12:33:08 | GET / HTTP/1.1                 |
| 4355 | 10.131.0.1 | [08/Nov/2017:12:33:08 | GET /login.php HTTP/1.1        |
| 4356 | 10.131.0.1 | [08/Nov/2017:14:14:05 | GET / HTTP/1.1                 |
| 4360 | 10.131.0.1 | [08/Nov/2017:17:28:06 | GET / 

| 9057 | 10.131.0.1 | [19/Dec/2017:14:20:37 | GET / HTTP/1.1           |

session no:  128
|      | IP         | Time                  | URL                            |
|------+------------+-----------------------+--------------------------------|
| 9061 | 10.131.0.1 | [19/Dec/2017:16:56:11 | GET / HTTP/1.1                 |
| 9062 | 10.131.0.1 | [19/Dec/2017:16:56:29 | GET /login.php HTTP/1.1        |
| 9063 | 10.131.0.1 | [19/Dec/2017:16:56:34 | GET /css/font-awesome.min.css  |
| 9064 | 10.131.0.1 | [19/Dec/2017:16:56:35 | GET /css/main.css HTTP/1.1     |
| 9065 | 10.131.0.1 | [19/Dec/2017:16:56:38 | GET /css/style.css HTTP/1.1    |
| 9066 | 10.131.0.1 | [19/Dec/2017:16:56:39 | GET /css/normalize.css HTTP/1. |

session no:  129
|      | IP         | Time                  | URL                            |
|------+------------+-----------------------+--------------------------------|
| 9074 | 10.131.0.1 | [19/Dec/2017:20:03:15 | GET /css/bootstrap.min.css HTT |

session no:  130
|   

|       | IP         | Time                  | URL                            |
|-------+------------+-----------------------+--------------------------------|
|  9500 | 10.131.0.1 | [29/Jan/2018:20:17:49 | GET /login.php HTTP/1.1        |
|  9504 | 10.131.0.1 | [29/Jan/2018:20:17:50 | GET /js/vendor/modernizr-2.8.3 |
|  9524 | 10.131.0.1 | [29/Jan/2018:20:17:53 | GET /home.php HTTP/1.1         |
|  9525 | 10.131.0.1 | [29/Jan/2018:20:17:53 | GET /login.php HTTP/1.1        |
|  9526 | 10.131.0.1 | [29/Jan/2018:20:17:53 | GET /js/vendor/modernizr-2.8.3 |
|  9529 | 10.131.0.1 | [29/Jan/2018:20:17:54 | GET /login.php HTTP/1.1        |
|  9537 | 10.131.0.1 | [29/Jan/2018:20:17:58 | GET /home.php HTTP/1.1         |
|  9538 | 10.131.0.1 | [29/Jan/2018:20:17:59 | GET /login.php HTTP/1.1        |
|  9557 | 10.131.0.1 | [29/Jan/2018:20:18:00 | GET /home.php HTTP/1.1         |
|  9558 | 10.131.0.1 | [29/Jan/2018:20:18:01 | GET /login.php HTTP/1.1        |
|  9560 | 10.131.0.1 | [29/Jan/2018:20:1

|       | IP         | Time                  | URL                            |
|-------+------------+-----------------------+--------------------------------|
| 15852 | 10.131.0.1 | [02/Mar/2018:04:30:47 | GET /login.php HTTP/1.1        |
| 15853 | 10.131.0.1 | [02/Mar/2018:04:30:47 | GET /css/bootstrap.min.css HTT |
| 15856 | 10.131.0.1 | [02/Mar/2018:04:30:47 | GET /css/main.css HTTP/1.1     |
| 15857 | 10.131.0.1 | [02/Mar/2018:04:30:47 | GET /css/style.css HTTP/1.1    |
| 15859 | 10.131.0.1 | [02/Mar/2018:04:30:47 | GET /js/vendor/jquery-1.12.0.m |
| 15861 | 10.131.0.1 | [02/Mar/2018:04:30:47 | GET /fonts/fontawesome-webfont |
| 15862 | 10.131.0.1 | [02/Mar/2018:04:30:52 | GET /login.php HTTP/1.1        |
| 15863 | 10.131.0.1 | [02/Mar/2018:04:30:53 | GET /login.php HTTP/1.1        |
| 15864 | 10.131.0.1 | [02/Mar/2018:04:30:54 | GET /login.php HTTP/1.1        |
| 15869 | 10.131.0.1 | [02/Mar/2018:04:47:48 | GET /css/normalize.css HTTP/1. |
| 15871 | 10.131.0.1 | [02/Mar/2018:04:4